In [113]:
import pandas as pd 

# Esteban: Cambiar la siguiente ruta a la tuya
ruta_datos_xl = "c:/_DATA/DW3/Muestra_BD_incidentes.xlsx"
df0 = pd.read_excel( ruta_datos_xl )

In [114]:
df0.columns

Index(['CODIGO', 'TIPO DE INCIDENTE', 'TIPO FINALIZACION', 'FECHA ALTA',
       'FECHA DE CREACION', 'FECHA EN EL LUGAR', 'FECHA DE FINALIZACION',
       'LATITUD', 'LONGITUD', 'COMUNA', 'TIENE AGENTE', 'AG1', 'AG2', 'AG3',
       'INTERVALO', 'DIRECCION', 'TURNO', 'VEHICULO INVOLUCRADO1',
       'VEHICULO INVOLUCRADO2', 'VEHICULO INVOLUCRADO3',
       'VEHICULO INVOLUCRADO4', 'VEHICULO SMM', 'MOTO SMM', 'BETA 1', 'BETA 2',
       'BETA 3', 'MOTO', 'PMV', 'UBICACIÓN PMV1', 'UBICACIÓN PMV2', 'CCTV',
       'UBICACIÓN CCTV1', 'UBICACIÓN CCTV2', 'NOTAS', 'TIEMPO DE CREACION',
       'TIEMPO DE RESPUESTA', 'TIEMPO DE ATENCION', 'TIEMPO TOTAL',
       'TIPO RECURSO', 'COD DIA', 'DIA', 'FECHA', 'HORA', 'MES', 'SEMANA',
       'MES NOM', 'TIPO', 'Enc'],
      dtype='object')

In [115]:
# Remover columnas calculadas 
df1 = df0.drop( ['TIEMPO DE CREACION', 'TIEMPO DE RESPUESTA', 'TIEMPO DE ATENCION', 'TIEMPO TOTAL', 
                  'TIPO RECURSO', 'COD DIA', 'DIA', 'FECHA', 'HORA', 'MES', 'SEMANA', 'MES NOM', 'TIPO', 'Enc'],
                axis=1)
# Remover otras columnas poco útiles
# Esteban: agregá las que querás
#  Las de información personal las podés dejar y cifrarlas más abajo
# Dejarlas permitiría responder preguntas como 'hubo alguna persona involucrada en más de N accidentes en el periodo dado?'
# sin revelar la identidad de la persona...
df1 = df1.drop( [ 'PMV', 'UBICACIÓN PMV1', 'UBICACIÓN PMV2',
                  'CCTV', 'UBICACIÓN PMV2', 'CCTV', 'UBICACIÓN CCTV1', 'UBICACIÓN CCTV2' ],
                axis=1 )

In [116]:
del df0 # limpieza de basura

In [85]:
from hashlib import md5
import hashlib
import base64
import math

def make_encrypter( long, salt = 'clave_secreta' ) :
    """construye una funcion encriptador y la retorna."""
    def encrypt( s_plano ) : 
        if (type(s_plano)==float and math.isnan(s_plano)) :
            return s_plano 
        
        s_plano = str( s_plano )
       
        if s_plano.strip() == ''  :
            return ''
        
        return base64.b64encode( md5( (s_plano + salt).encode("utf8") )
                                 .digest() )[ :long].decode('ascii')
    
    encrypt.__doc__ =  f"""Esta función recibe una cadena (unicode) no vacía, la concatena con '{salt}'
          y retorna un md5 codificado en b64 y truncado a longitud {long}. 
          Cadenas vacías y NaNs no se cifran"""
        
    return encrypt #retorna (referencia a) la funcion que acabamos de definir dinámicamente
        

In [86]:
#%% ejemplo de uso 

encrypt = make_encrypter( long=5, salt='Mateo')
help( encrypt)

Help on function encrypt in module __main__:

encrypt(s_plano)
    Esta función recibe una cadena (unicode) no vacía, la concatena con 'Mateo'
    y retorna un md5 codificado en b64 y truncado a longitud 5. 
    Cadenas vacías y NaNs no se cifran



In [87]:
encrypt( 'TT-827' )

'HEZuw'

In [88]:
# ESTEBAN: Agregar aquí otras que se quieran encryptar aquí columnas aquí
# cambiar el valor de SALT por una cadena secreta tuya arbitraria, para que no sea posible revertir el cifrado

SALT = 'Mateo' 

cols_a_cifrar = [ ('AG1', 6), ('AG2', 6)]
dfc = df1.copy() 

for col, long  in cols_a_cifrar : 
    encrypt = make_encrypter( long, salt=SALT )
    dfc[ col ] = df1[col].apply( encrypt )
    
    

In [89]:
dfc[['CODIGO', 'TIPO DE INCIDENTE', 'TIPO FINALIZACION', 'AG1', 'AG2']]

,CODIGO,TIPO DE INCIDENTE,TIPO FINALIZACION,AG1,AG2
0,20180101001435000,002 Lesiones accidente de Tránsito (979),Negativo en sitio,BXiAhn,9b4/Bs
1,20180101001601000,007 Servicio Recoge Motos,Positivo,ozqRU5,NaN
2,20180101002501000,002 Lesiones accidente de Tránsito (979),Positivo,BXiAhn,9b4/Bs
3,20180101004420000,002 Lesiones accidente de Tránsito (979),Positivo,FKAoZS,NaN
4,20180101005604000,002 Lesiones accidente de Tránsito (979),Negativo en sitio,psQHFn,NaN
5,20180101011151000,002 Lesiones accidente de Tránsito (979),Conci. en el sitio,L2fm/w,crq08/
6,20180101012650000,002 Lesiones accidente de Tránsito (979),Negativo en sitio,H+s90O,9YMOJg
7,20180101013331000,002 Lesiones accidente de Tránsito (979),Positivo,psQHFn,NaN
8,20180101013742000,002 Lesiones accidente de Tránsito (979),Positivo,YQKP+I,NaN
9,20180101020155000,002 Lesiones accidente de Tránsito (979),Conci. en el sitio,L2fm/w,kL3kW6


In [ ]:
# Limpieza de basura
del df0
del df1

In [112]:
# Esteban: poner tu propia ruta a un excel aquí, pero dejar la extensión h5.
# hdf5 es un formato binario open source para guardar información tabular de alta densidad. 
dfc.to_hdf( 'c:/_DATA/DW3/incidentes.h5', complevel=5, key='data' )

C:\Users\mrestrepo\AppData\Local\Continuum\miniconda3\lib\site-packages\pandas\core\generic.py:1996: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed-integer,key->block1_values] [items->['TIPO DE INCIDENTE', 'TIPO FINALIZACION', 'COMUNA', 'TIENE AGENTE', 'AG1', 'AG2', 'AG3', 'INTERVALO', 'DIRECCION', 'VEHICULO INVOLUCRADO1', 'VEHICULO INVOLUCRADO2', 'VEHICULO SMM', 'MOTO SMM', 'BETA 1', 'BETA 2', 'MOTO', 'NOTAS']]

  return pytables.to_hdf(path_or_buf, key, self, **kwargs)
